https://chromedriver.chromium.org/downloads

In [ ]:
#第一次run
%pip install ipywidgets
%pip install IPython
%pip install selenium

In [27]:
import ipywidgets as widgets
from IPython.display import display
import os, re, shutil

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from xml.dom import minidom

driver = webdriver.Chrome()
url = "https://yqnn.github.io/svg-path-editor/"

driver.get(url)

In [86]:
input_folder = widgets.Text()
display(input_folder)

base_path = '/Users/willchan/Desktop/review/3_Svg_decompose_from_machine/' #本身個folder係邊
file_no = ""
org_file = []
stroke_no = 0

paste_flag = False

last_stroke = ""
cur_stroke = ""
next_stroke = ""

# input_order = widgets.Text()
# display(input_order)

pre = f"{file_no}_"

# new_folder_location = "/Users/willchan/Desktop" #想擺邊
new_order_folder = '/Users/willchan/Desktop/review/3_Svg_decompose_from_machine_fixed/'

try:
    os.mkdir(new_order_folder)
except:
    pass

def extend(input_list: str):
    split_string = input_list.split()
    temp_list = []

    for substring in split_string:
        if '-' in substring:
            substring_split = substring.split('-')
            start = int(substring_split[0])
            end = int(substring_split[1])
            temp_list.extend(range(start, end+1))
        else:
            temp_list.append(int(substring))
    
    return temp_list  

def reset():
    global new_folder_location, new_order_folder, base_path, file_no, org_file, pre
    new_order_folder = '/Users/willchan/Desktop/review/3_Svg_decompose_from_machine_fixed/'

    base_path = '/Users/willchan/Desktop/review/3_Svg_decompose_from_machine/' #本身個folder係邊
    file_no = ""
    org_file = []
    pre = f"{file_no}_"

    input_folder.value = ""
    

def print_values(sender):
    global file_no, base_path, org_file, stroke_no
    print(sender.value)
    temp = sender.value.split()
    
    file_no = temp[0]
    stroke_no = int(temp[1])
    
    base_path += file_no
    print(base_path)
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file == ".DS_Store" or 'trace' in file or 'turn' in file:
                continue
            org_file.append(file)
    org_file = sorted(org_file, key=lambda x:int(x.split("_")[1]))
    print(org_file)

    send_svg(get_svg_path(org_file[stroke_no]))
    # reset()


def new_order(sender):
    # order = sender.value.split(" ")
    order = extend(sender.value)
    # print("order" , order) 
    l = len(order)
    if l != len(org_file):
        raise Exception("Length Not match")
    
    # for i in range(l):
    #     if not order[i].isdigit():
    #         raise Exception(f"{order[i]} not a digit")
    #     if str(i) not in order:
    #         # print(i)
    #         raise Exception(f"{i} not in the list")

    os.mkdir(os.path.join(new_order_folder, file_no))

    for i in range(len(org_file)):
        # if order[i] == '-':
        #     continue
        suffix = (re.search(r'_[A-Z](.*)', org_file[i])).group()
        org_path = f'{base_path}/{org_file[int(order[i])]}'
        # print(order[i])
        target_path = f'{new_order_folder}/{file_no}/{file_no}_{i}{suffix}'

        # print(org_path)
        shutil.copyfile(org_path, target_path)
        
    print("done")
    reset()


def wait_send(by, path, send):
    WebDriverWait(driver, 180).until(EC.presence_of_element_located((by, path)))
    driver.find_element(by, path).send_keys(send)

def send_svg(svg: str):
    WebDriverWait(driver, 180).until(EC.presence_of_element_located((By.CSS_SELECTOR,"textarea")))
    driver.find_element(By.CSS_SELECTOR,"textarea").clear()
    driver.find_element(By.CSS_SELECTOR,"textarea").send_keys(svg)

    # wait_send(By.CSS_SELECTOR,"textarea", svg)
    # driver.execute_script("document.querySelector('.drawings.ng-tns-c94-0').style.transform='rotateX(180deg)'")

def get_svg_path(file):
    path = f"{base_path}/{file}"

    doc = minidom.parse(path)  # parseString also exists
    # print(doc.getElementsByTagName("ns0:path"))
    path_strings = [path.getAttribute('d') for path
                    in doc.getElementsByTagName('ns0:path')]
    doc.unlink()
    return (re.sub(r'([A-Y])', r'\1 ', re.sub(r'(\d{3})(Q\d{3})', r'\1 \2' ,path_strings[0])))
    


input_folder.on_submit(print_values)

# input_order.on_submit(new_order)



Text(value='')

/var/folders/y7/ytwy5wl55bs956vg5hlz1n1r0000gn/T/ipykernel_26904/2404818165.py:129: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  input_folder.on_submit(print_values)


31179 2
/Users/willchan/Desktop/review/3_Svg_decompose_from_machine/31179
['31179_0_P.svg', '31179_1_H.svg', '31179_2_S.svg', '31179_3_P.svg', '31179_4_D.svg', '31179_5_D.svg', '31179_6_P.svg', '31179_7_P.svg', '31179_8_N.svg']


In [101]:

if paste_flag == False:
    cur_stroke = driver.find_element(By.CSS_SELECTOR,"textarea").get_attribute('value') if cur_stroke == "" else cur_stroke

l = len(org_file)

if paste_flag == False:
    paste_flag = True
    last_stroke = get_svg_path(org_file[stroke_no-1]) if last_stroke == "" else last_stroke
    next_stroke = get_svg_path(org_file[stroke_no+1]) if next_stroke == "" else next_stroke
    if stroke_no + 1 < l:
        send_svg(last_stroke+cur_stroke+ next_stroke)
        
    else:
        send_svg(last_stroke+get_svg_path(org_file[stroke_no]) )
    
else:
    paste_flag = False
    cur_stroke = driver.find_element(By.CSS_SELECTOR,"textarea").get_attribute('value').replace(last_stroke,"").replace(next_stroke,"")
    send_svg(cur_stroke)

In [24]:
#改完run
def change_svg_path(file):

    changed_path = driver.find_element(By.CSS_SELECTOR,"textarea").get_attribute('value')

    path = f"{base_path}/{file}"

    # doc = minidom.parse(path) 
    # # print(doc.getElementsByTagName("ns0:path"))
    # path_strings = [path.getAttribute('d') for path
    #                 in doc.getElementsByTagName('ns0:path')]
    # doc.unlink()

    doc = minidom.parse(path)

    nodes = doc.getElementsByTagName('ns0:path')

    for node in nodes:
        # 修改節點的屬性
        node.setAttribute('d', changed_path)

        # 取得節點的屬性值
        # attribute_value = node.getAttribute('d')

    with open(path, 'w') as f:
        doc.writexml(f)
    temp_new_folder = new_order_folder + file_no
    if os.path.isdir(temp_new_folder):
        shutil.rmtree(temp_new_folder)
    shutil.copytree(base_path ,temp_new_folder)

    reset()

change_svg_path(org_file[stroke_no])